# Data Science Project SoSe 2024
## Team 07
- Maximilian Hoffmann
- Kilian Kempf
- Daniel Schneider
- Tom Schuck

## Project Submission

## Notebook content

- Data Initialization
- General Analysis
- Feature Engineering
- Feature Analysis
- Model Training
- Model Evaluation
- Accuracy Prediction

In [ ]:
import os
import pandas as pd
from data_management import DataManager

## Data Initialization

In [ ]:
DATA_DIR = os.path.join(os.getcwd(), 'data/Instacart')

op_prior = pd.read_csv(os.path.join(DATA_DIR, 'order_products__prior.csv.zip'))
op_train = pd.read_csv(os.path.join(DATA_DIR, 'order_products__train.csv.zip'))

tip_train = pd.read_csv(os.path.join(DATA_DIR, 'tip_trainingsdaten1_.csv'))[['order_id', 'tip']]
tip_test = pd.read_csv(os.path.join(DATA_DIR, 'tip_testdaten1_template.csv'))

orders = pd.read_csv(os.path.join(DATA_DIR, 'orders.csv.zip'))
aisles = pd.read_csv(os.path.join(DATA_DIR, 'aisles.csv.zip'))
departments = pd.read_csv(os.path.join(DATA_DIR, 'departments.csv.zip'))
products = pd.read_csv(os.path.join(DATA_DIR, 'products.csv.zip'))

data_manager = DataManager(op_prior, op_train, tip_train, tip_test, orders, products, aisles, departments)
order_amount = len(data_manager.get_orders_tip())

## Data Analysis
Several Analysis were done to find any tip patterns in the data. The purpose of this analysis is to find specific features that are relevant for the prediction of tip. To find out which attributes have a significant influence on the tip, several analysis task were executed.

In [ ]:
from analysis import Analysis, HourOfDay, DayOfWeek, Department, ReorderedAnalysis, DaysSincePriorOrder, AssocRules, TipSequence, OrderNumber, Aisle, Product, NumberOrderUser, ProductCardOrder, GeneralAnalysis, GeneralFacts

TODO: Describe here your analysis task. What is the analysis about, any assumptions before? What did we expect?
TODO: Describe the results also here in this Markdown



##### Analysis Task Name
**Analysis:** 

**Result Interpretation:**

In [1]:
# TODO: Initialize your analysis instance
# TODO: Produce your output

##### General Facts
**Analysis:** For data understanding and getting familiar with the data set and the overall relations between the main entities, some general facts about the dataset were collected. Therefore several facts has been calculated containing the different entities and there relations.

**Result Interpretations:** The results are important to clarify weather the calculations fit with the overall facts of the dataset and do not contain any calculation errors or changed the dataset.

In [ ]:
general_facts = GeneralFacts(data_manager)
general_facts.execute_analysis()

##### Add to card order analysis per Product
**Analysis:** To find out weather the add to card order of a specific product has an impact on the tip probability of that product, an analysis task has been created which calculates the tip probability of the products in dependence of the add_to_cart_order.

**Result Interpretations:** In the boxplot below, we see ...... 
&rarr; **No overall impact of add_to_card_order on the tip probability of the products** &rarr; no feature crafted

## Feature Engineering
With the results of the general analysis task and our specific assumptions of any relation between tip and various data combinations, a few features were build that we think will make a potential impact. The explanation of each feature and corresponding impact analysis will be covered in the next chapter (Feature analysis). To execute the impact analysis of the features, it is necessary to compute them all before.

For the computation of the features, it is distinguished between two types of features:
- **Static features:** Features that are calculated user specific and consider just the last orders of the specific user. These features can be computed once and do not need to be recalculated for a newly created fold in a time series cross validation.
- **Dynamic features:** Features that are calculated user comprehensive like the product tip rate of all orders. For these features we use to data of the whole dataset or fold in cross validation. Therefore the dynamic features need to be recalculated for every new fold in the cross validation.

In [ ]:
from feature_engineering.static_features import ContainsAlcohol, DowHighTipProbability, TipHistory, OrderSize, OrderNumberSquared, PrevOrderTipped, MeanOrderedRate, CustomerLifetime, ReorderedRatio, OrderFrequency, HodHighTipProbability, PrevTippedProductsRatio, RelDaysSinceTip, DaysSinceTip, LastTipSequence, AvgSizePrevOrders, SimOrdersTipRatio
from feature_engineering.dynamic_features import ProductTipRate, DepartmentTipRate, AisleTipRate, AssocRulesAisles, AssocRulesDepartments

In [ ]:
# Static Features
tip_history = TipHistory()
reordered_rate = ReorderedRatio()
order_size = OrderSize()
prev_tipped_products_ratio = PrevTippedProductsRatio()
customer_lifetime = CustomerLifetime()
prev_order_tipped = PrevOrderTipped()
order_frequency = OrderFrequency()
mean_ordered_rate = MeanOrderedRate()
rel_days_since_tip = RelDaysSinceTip()
days_since_tip = DaysSinceTip()
sim_orders_tip_ratio = SimOrdersTipRatio()
product_tip_rate = ProductTipRate()
order_number_squared = OrderNumberSquared()
hod_high_tip_probability = HodHighTipProbability()
dow_high_tip_probability = DowHighTipProbability()
contains_alcohol = ContainsAlcohol()
avg_size_prev_orders = AvgSizePrevOrders()

# Dynamic Features
department_tip_rate = DepartmentTipRate()
aisle_tip_rate = AisleTipRate()
last_tip_sequence = LastTipSequence()
assoc_rules_departments = AssocRulesDepartments()
assoc_rules_aisles = AssocRulesAisles()

In [ ]:
# Static Features
data_manager.register_feature(tip_history)
data_manager.register_feature(reordered_rate)
data_manager.register_feature(order_size)
data_manager.register_feature(customer_lifetime)
data_manager.register_feature(prev_order_tipped)
data_manager.register_feature(prev_tipped_products_ratio)
data_manager.register_feature(order_frequency)
data_manager.register_feature(sim_orders_tip_ratio)
data_manager.register_feature(mean_ordered_rate)
data_manager.register_feature(last_tip_sequence)
data_manager.register_feature(rel_days_since_tip)
data_manager.register_feature(days_since_tip)
data_manager.register_feature(order_number_squared)
data_manager.register_feature(hod_high_tip_probability)
data_manager.register_feature(dow_high_tip_probability)
data_manager.register_feature(contains_alcohol)
data_manager.register_feature(avg_size_prev_orders)

# Dynamic Features
data_manager.register_feature(product_tip_rate)
data_manager.register_feature(department_tip_rate)
data_manager.register_feature(aisle_tip_rate)
data_manager.register_feature(assoc_rules_aisles)
data_manager.register_feature(assoc_rules_departments)

Because of the computing time of each feature, the calculation has been done once before and is imported to the local file system. If it is required to compute the features once again, the next two lines of code need to be executed.

In [ ]:
# In case a recomputing of the features is necessary. Remove the comments for the next two lines of code.
# data_manager.compute_features()
# data_manager.export_features('data/prepared_data/computed_features.csv.zip', only_static=False)

data_manager.import_features('data/prepared_data/computed_features.csv.zip', only_static=False)

## Feature analysis
To decide weather a feature will take place in the feature set of the machine learning model, it is necessary to analyze which features have a significant impact on tip. All features that has been computed are described and analyzed in the following section.


##### Feature_Name
**Description:** Describe how the feature is calculated and what does it mean.

**Analysis:** Which impact does the feature have on tip. Does it have any cocorrelations to other features
&rarr; **Is the feature part of the model feature set? and why (one sentence)**

In [ ]:
# TODO: Execute the feature_name.analyze_feature() method

### Comparison of features

Compare correlations
Compare coefficients

## Model Training

Describe Pipeline Architecture. Time seroius cross validation 


## Accuracy Prediction